# Images in Python

**Part of the IAFIG-RMS *Python for Bioimage Analysis* Course.**

*Dr Chas Nelson*

2019-12-09 1000--1200

## Aim

To revise key concepts of digital images and make connections to counterpart concepts in Python.

*Note: We will go through a lot of concepts quite quickly in this session. However, you should know all of them and the point of this session is to make links between what you know and introduce you to Python packages and conventions that we will use throughout the course.*

## ILOs

* Describe how NumPy arrays and dtypes relate to features of digital images
* Be able to read/write multidimensional image stacks to/from NumPy arrays and multi-page TIFF files
* Be able to subsample your image using NumPy array smart indexing
* Extract and plot a histogram of a multidimensional image

## Digital Images and NumPy Arrays

### What is a Digital Image?

* Digital images are arrays or matrices of pixels (picture elements), i.e. numbers, that represent an object or scene.
  * Unless otherwise states we will assume these numbers are integers.
* In a grayscale image, each of these numbers is the intensity.
* A grayscale image usually represents a narrow band of wavelengths, i.e. a colour or channel.
  * Multiple grayscale images together can form a colour image, e.g. RGB.

![Digital images are arrays or matrices of pixels (or voxels) - i.e. numbers.](./assets/digitalimage.png)

*Courtesy of Dominic Waithe; apple from https://www.wikipiedia.org*

* We can navigate a 2D image by using row and column numbers to extract a single pixel value.
  * People use two coordinate systems for this: $i$ & $j$ or $x$ and $y$.
  * We will use $x$ for rows and $y$ for columns throughout this course. $x=0, y=0$ is the top left corner of the image.
  * (We will introduce higher dimensions as and when necessary.)
  
![Acessing pixels using axes.](./assets/arrays.png)

*Adapted from https://github.com/elegant-scipy/elegant-scipy*

### What is a NumPy Array?

* In Python, NumPy Arrays are arrays or matrices of numbers, as such we can use arrays to represent digital images.
* For the representation of a grayscale image, we would use an array with as many rows and columns as the each of these numbers in the grayvalue.
* We can navigate an array by using row and column numbers to extract a single element - this is called indexing.
  * Don't forget that Python starts counting at $0$ and not at $1$
  * As with digital images, we will use $x$ for rows and $y$ for columns throughout this course. $x=0, y=0$ is the top left corner of the array.
  * Note: Python and FIJI swap $x$ and $y$, so the data will be transposed.

In [ ]:
import numpy as np

# Create an array of a Guassian disc with values between 0 and 8
# The array has ten rows and fitteen columns
# Don't worry too much about this code - it just creates a toy example

rows = 10
columns = 15
x, y = np.meshgrid(np.linspace(-1,1,columns), np.linspace(-1,1,rows))
d = np.sqrt(x*x+y*y)
sigma = 0.5
myGaussianDisc = (8*np.exp(-( (d)**2 / ( 2.0 * sigma**2 ) ) )).astype('uint8')

display(myGaussianDisc)

In [ ]:
# Extract the element value at x=5 and y=6
x = 5
y = 4
display(myGaussianDisc[x,y])

In [ ]:
# Import two key plotting/display packages
%matplotlib widget
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

In [ ]:
f,axes = plt.subplots(1,2)
(aMPL, aSNS) = axes.flatten()

# Display array as grayscale image
# Note that this autoscales (like ImageJ).
# This doesn't affect the underlying array
aMPL.imshow(myGaussianDisc, cmap="gray")  # We discuss colourmaps (cmaps, LUTs) below
aMPL.set_axis_off()  # This turns axes off
aMPL.set_title("grayscale Image...")

# Display array as heatmap
# Note that this also autoscales.
sns.heatmap(myGaussianDisc,cmap="viridis",square=True,annot=True,ax=aSNS)
aSNS.set_ylim(0,10)  # Make sure we can see the full array
aSNS.invert_yaxis()  # Heatmap and imshow flipx, let's undo that
aSNS.set_ylabel('X')  # Note that x and y are swapped for display (compared to indexing and FIJI)
aSNS.set_xlabel('Y')  # Note that x and y are swapped for display (compared to indexing and FIJI)
aSNS.set_title("... as Heatmap")

plt.show()

### Colour Digital Images

* Colour images have multiple 'channels.
* In photography and monitors/screens three such channels represent RGB.
* In microscopy we can have any number of channels, each representing a different spectral band, i.e. a different fluorphore.

### 'Colour' NumPy Arrays?

* NumPy arrays can have more than two dimensions ($x$ and $y$).
* If you're doing a lot of computation, it's conventional to represent channels in the first dimension, as this is quicker for access.
  * $x$ and $y$ then become the second and third dimensions, respectively.
* However, `matplotlib` expects colour to be the third dimension.
  * $x$ and $y$ are then set to the first and second dimension, respecitvely.
* We will use `matplotlib`'s convention for 2D images with colour channels.

In [ ]:
# Create a 3-channel array of offset Gaussian discs with values  between 0 and 8 (exclusive)
# The array has ten rows and fifteen columns

rows = 10
columns = 15
x, y = np.meshgrid(np.linspace(-1,1,columns), np.linspace(-1,1,rows))
d = np.sqrt(x*x+y*y)
sigma = 0.5
disc = (8*np.exp(-( (d)**2 / ( 2.0 * sigma**2 ) ))).astype('uint')
myRGBArray = np.stack([disc,np.roll(disc,2,axis=0),np.roll(disc,2,axis=1)],axis=2)

print("Red:")
display(myRGBArray[:,:,1])

In [ ]:
f, axes = plt.subplots(2,2)  # Create four subplots (2x2 grid)
(aR, aG, aB, aRGB) = axes.flatten()

aR.imshow(myRGBArray[:,:,0], cmap="gray")  # Displaying individual channels in gray prevents false highlighting of areas
aR.set_axis_off()
aR.set_title("Red Channel")

aG.imshow(myRGBArray[:,:,1], cmap="gray")
aG.set_axis_off()
aG.set_title("Green Channel")

aB.imshow(myRGBArray[:,:,2], cmap="gray")
aB.set_axis_off()
aB.set_title("Blue Channel")

aRGB.imshow(myRGBArray/8)  # Matlab expects values between 0 and 1 or 0 and 255 (see bit-depth below)
aRGB.set_axis_off()
aRGB.set_title("Composite")

plt.show()

### Spatial Sampling

* The number of pixels in in a image denote the resolution.
  * In NumPy, the resolution is the shape of the array, e.g. `myArray.shape`
* Each pixel will represent a physical size, e.g. $3 \mu m \times 3 \mu m$ (also often call resolution)
  * This is not explicitly coded into a digital image or NumPy array so we will need to be extracted from the metadata.

<!-- ![The number of pixels in in a image denote the resolution.](./assets/resolution.png) -->

### Intensity Sampling

* The bit-depth of an image describes the dynamic range of a pixel, i.e. the difference between the minimum and maximum possible values.
  * The equivalent feature of a Numpy array is its `dtype`.
* Most DSLR cameras will use 8-bit for grayscale images and 3x8-bit (24-bit) for colour images.
  * In NumPy this is `uint8`.
* Most scientific cameras will use 12-bit or 16-bit. (Both will appear to be 16-bit due to the container files.)
  * In NumPy this is `uint16`.
* A lot of image processing requires continuous pixel values, i.e. decimals between 0 and 1.
  * In NumPy these values are `float`.
* Up to 16-bit, computers use unsigned integers to represent pixel values; however, 32 and 64-bit images will likely use `float` (continous numbers with decimal points). Each has its benefits.
* It is always possible to process images in a higher bit-depth container/array without lose of data, e.g. converting an 8-bit image to a `float` array; but be wary of accidentally doing things the other way, e.g. opening a 16-bit image in an `uint8` array.

![The bit-depth of an image describes the dynamic range of a pixel, i.e. the difference between the minimum and maximum pixel values.](./assets/bitdepth.png)

*Courtesy of Dominic Waithe; apple from https://www.wikipedia.org*

### Look-Up Tables (LUTs) and Composites

* Pixel values can be visualised with a range of different colours.
* The 'mapping' from a grayscale value to a colour is done through a Look-Up Table (LUT)
* It's important to be aware that there are good and bad LUTs - some, like Jet, are not perceptually uniform and will artificially highlight the yellow-red features.
  * Poor use of mapping can create perceptual changes that are independent of the underlying pixel value

In [ ]:
# Create an array of a Guassian disc with values between 0 and 4096 (exclusive)
# The array has ten rows and fifteen columns

rows = 10
columns = 15
x, y = np.meshgrid(np.linspace(-1,1,columns), np.linspace(-1,1,rows))
d = np.sqrt(x*x+y*y)
sigma = 0.5
my12BitArray = ((2**12-1)*np.exp(-( (d)**2 / ( 2.0 * sigma**2 ) ) )).astype('uint16')  # note how 12 bits are stored in a 16-bit array

f, axes = plt.subplots(2,3)  # Create six subplots (2x3 grid)
(aG, aS, aV, aH, aRB, aHSV) = axes.flatten()

aG.imshow(my12BitArray, cmap="gray", vmin=0, vmax=(2**16)-1)
aG.set_axis_off()
aG.set_title("Gray (not scaled)")

aS.imshow(my12BitArray, cmap="gray")
aS.set_axis_off()
aS.set_title("Gray (scaled)")

aV.imshow(my12BitArray, cmap="viridis")  # also check out 'magma'
aV.set_axis_off()
aV.set_title("Viridis (perceptually uniform)")

aH.imshow(my12BitArray, cmap="hot")
aH.set_axis_off()
aH.set_title("Hot (squential colormap)")

aRB.imshow(my12BitArray, cmap="RdBu")
aRB.set_axis_off()
aRB.set_title("RdBu (diverging colormap)")

aHSV.imshow(my12BitArray, cmap="hsv")
aHSV.set_axis_off()
aHSV.set_title("HSV (not perceptually uniform)")

plt.show()

* Different channels/colours of an image can be combined to create a composite image.
* We're all used to seeing RGB composite images everyday.
* However, in microscopy, we can choose the LUTs used for independent channels giving us greater flexibility.
* Bear in mind that some colours, e.g. the ubiquitous red-green pairing, are not colourblind friendly.
  * To get around this, we recommend always showing invidiual channels in grayscale and, for composites, using green-magenta or green-cyan colour pairs.

In [ ]:
# Create an array of offset Guassian discs with values between 0 and 1024 (exclusive)
# The array has ten rows and fifteen columns and two channels

rows = 10
columns = 15
x, y = np.meshgrid(np.linspace(-1,1,columns), np.linspace(-1,1,rows))
d = np.sqrt(x*x+y*y)
sigma = 0.5
disc = ((2**8-1)*np.exp(-( (d)**2 / ( 2.0 * sigma**2 ) ))).astype('uint8')
my2ChannelArray = np.stack([disc,np.roll(disc,1,axis=0)],axis=2)

f, axes = plt.subplots(1,3)  # Create three subplots (1x3 grid)
(c1, c2, comp) = axes.flatten()

c1.imshow(my2ChannelArray[:,:,0], cmap="gray")
c1.set_axis_off()
c1.set_title("Channel 1")

c2.imshow(my2ChannelArray[:,:,1], cmap="gray")
c2.set_axis_off()
c2.set_title("Channel 2")

# Make a green-magenta mapping of the two channels and move colour to end dimension
green_magenta = np.moveaxis([my2ChannelArray[:,:,1],my2ChannelArray[:,:,0],my2ChannelArray[:,:,1]],0,-1)

comp.imshow(green_magenta)
comp.set_axis_off()
comp.set_title("Composite")

plt.show()

### Working in More Than Two Dimensions

* Python is able to use arrays of N dimensions.
* In microscopy, we're likely to need up to 6: $x$, $y$, $z$, $t$ and $\lambda$.
* For computational efficiency, in multidimensional scenarios, we will treat the first axis for time, the second for z-slice the third as $x$, the fourth as $y$, and the fifth as colour/wavelength ($\lambda$). (The last three dimensions are as previously described.)
* You may choose a different dimension order depending on your analysis needs, e.g. if you are treating each channel differently then it make make more sense for $\lambda$ to be the first dimension.

![Python can cope with arryas that have N dimensions.](https://fgnt.github.io/python_crashkurs_doc/_images/numpy_array_t.png)

*From https://github.com/elegant-scipy/elegant-scipy*

In [ ]:
# Create an array of a Gaussian disc with values between 0 and 1024 (exclusive)
# The array has ten rows and fifteen columns and five z-slices

# Create a 3-channel array of offset Gaussian discs with values  between 0 and 8 (exclusive)
# The array has ten rows and fifteen columns

rows = 10
columns = 15
slices = 5
z, x, y = np.meshgrid(np.linspace(-1,1,slices), np.linspace(-1,1,rows), np.linspace(-1,1,columns), indexing='ij')
d = np.sqrt(x**2+y**2+z**2)
sigma = 0.5
my3DArray = ((2**8-1)*np.exp(-( (d)**2 / ( 2.0 * sigma**2 ) ))).astype('uint8')

display(my3DArray.shape)

In [ ]:
# Plot X-MIP, Y-MIP and Z-MIP using Matplotlib
f, axes = plt.subplots(2,2, gridspec_kw={'height_ratios':[rows/(rows+slices),slices/(rows+slices)],
                                         'hspace':0.5,
                                         'width_ratios':[slices/(columns+slices),columns/(columns+slices)]})  # Create four subplots (2x2 grid), grispec_kw is used to keep subplots proportional
# note: the current Jupyter Lab widget for matplotlib likes to stretch figures across the viewer width - hence the big gap between the left and right
(XZ, XY, null, ZY) = axes.flatten()
f.suptitle("Maximum Intensity Projection...")

ZY.imshow(my3DArray.max(axis=1), aspect=1.0, cmap="gray", interpolation='none')  # aspect can be set to the ratio between the z and y pixel size
ZY.grid(False)
# ZY.set_ylim(0,10)  # This and the next line are just a hacky fix to keep proportions the same for all subplots
# ZY.set_yticks(np.arange(0,slices))
ZY.set_title("...over X")

XZ.imshow(my3DArray.max(axis=2).T, aspect=1.0, cmap="gray", interpolation='none')  # aspect can be set to the ratio between the x and z pixel size
XZ.grid(False)
XZ.set_title("...over Y")

XY.imshow(my3DArray.max(axis=0), aspect=1.0, cmap="gray", interpolation='none')  # aspect can be set to the ratio between the x and y pixel size
XY.grid(False)
XY.set_title("...over Z")

null.set_axis_off()  # clear unused subplot

# Challenge: have a think about the intensity scaling that's going on here, how could that be accounted for?

plt.show()

In [ ]:
# Now use ipywidgets slider to scroll through the z-slices
import ipywidgets as widgets
import IPython.display as ipyd

#  Function to update slice on widget change
def update_slice(value):
    ax.imshow(my3DArray[value['new'],:,:], cmap='gray', vmin=my3DArray.min(), vmax=my3DArray.max(), interpolation='none')
    slice_label.value = 'Currently viewing slice {0} of {1}'.format(slice_slider.value,my3DArray.shape[0])

# Define starting plot
f, ax = plt.subplots(1,1)  # Create one subplot (1x1 grid)
ax.grid(False)
ax.set_title('Z-slice Viewer')
ax.imshow(my3DArray[0,:,:], cmap='gray', vmin=my3DArray.min(), vmax=my3DArray.max(), interpolation='none')

# Define widget
slice_slider = widgets.IntSlider(0, min = 0, max = my3DArray.shape[0])
slice_label = widgets.Label(value = 'Currently viewing slice {0} of {1}'.format(slice_slider.value,my3DArray.shape[0]))

slice_slider.observe(update_slice, names = 'value')

plt.show()
ipyd.display(slice_slider, slice_label)

## Reading/Writing Images in Python

### Data

The image we will use for the rest of this tutorial is from the Broad Bioimage Benchmark Collection data set BBBC0034v1 (https://data.broadinstitute.org/bbbc/; Thirstrup et al. 2018).

See https://data.broadinstitute.org/bbbc/BBBC034/ for the full description; however, the key points are:

* $1024 \times 1024 \times 52$ pixels
* $65 \times 65 \times 290$ nm/pixel
* 4 channels (each stored as separate files):
  * Cell membrane label (C=0)
  * Actin label (C=1)
  * DNA label (C=2)
  * Brightfield image (C=3)

In [ ]:
# Run this cell to download and unzip the images we need
# You do not need to understand it, but happy to explain during coffee
# You only need to run this cell once and then you may comment it out
import urllib.request
import io
import zipfile
import os
from tqdm import tqdm_notebook, tnrange

url = 'https://data.broadinstitute.org/bbbc/BBBC034/BBBC034_v1_dataset.zip'

with urllib.request.urlopen(url) as response:
    print("Downloading...")
    length = int(response.getheader('content-length'))
    chunk = max(4096, length//9999)
    
    buffer = io.BytesIO()
    size = 0
    for b in tnrange(length//chunk + 1):
        block = response.read(chunk)
        if not block:
            print("Finished reading after {0}% of file.".format(size/length))
        buffer.write(block)
        size = size + len(block)
    print("Finished reading file.")
    
    print("Unzipping... ",end="")
    zf = zipfile.ZipFile(buffer)
    os.makedirs('./assets/bbbc034v1/',exist_ok=True)
    zf.extractall(path='./assets/bbbc034v1')
    print("Complete.")

### Reading Multidimensional TIF files

We will use the [`tifffile` plug-in](https://www.lfd.uci.edu/~gohlke/code/tifffile.py.html) for reading and writing multidimensional TIF files. This is includedas part of the [`io` submodule](https://scikit-image.org/docs/stable/api/skimage.io.html) within the popular image processing module [`scikit-image`](https://scikit-image.org).

The `imread` function is able to deal with any multidimensional TIF file that `tiffile` is capable of reading - this means pretty much any TIF that's compatabile with ImageJ/FIJI.

#### Aside: Reading 'Collections'

* A 'collection' is a series of files containing a single 2D image whose names follow a name pattern, e.g. `file1.tif`, `file2.tif`, `file3.tif` all follow the pattern `file*.tif`.
* Collections may represent a time series, where each individual file could be a 2D TIF file, or a z-stack.
* One benefit with collections is that `scikit-image` can load frames into memory only when needed.
* Collections can read using, for example,

```python
from skimage import io

collection = io.imread_collection('./file*.tif',conserve_memory=True)
frame1 = collection[0]
```

In [ ]:
from skimage import io

# Read a single multidimensional TIF file, in this case a single channel with multiple z-slices.
c1 = io.imread('./assets/bbbc034v1/AICS_12_134_C=1.tif')

# Confirm that this image is in fact a numpy array
display(type(c1))

# Metadata for future use later
x_pixel_size = 65  # nm
y_pixel_size = 65  # nm
z_pixel_size = 290  # nm

<div style="background-color:#abd9e9; border-radius: 5px; padding: 10pt"><strong>Task:</strong> In the cell below confirm the size of these images in pixels and the bit-depth of the image.</div>

<div style="background-color:#abd9e9; border-radius: 5px; padding: 10pt"><strong>Task</strong> Using the codes above, create a new code cell and make a figure showing maximum projections in all three dimensions.</div>

<div style="background-color:#abd9e9; border-radius: 5px; padding: 10pt"><strong>Task</strong> Using the codes above, create a new code cell and make an interactive figure allowing you to scroll through $z$.</div>

<div style="background-color:#abd9e9; border-radius: 5px; padding: 10pt"><strong>Task</strong> Using the information above, read the other three channels for this data into the variables <code>c0</code>, <code>c2</code> and <code>c3</code>.</div>

<div style="background-color:#abd9e9; border-radius: 5px; padding: 10pt"><strong>Task</strong> Use your existing Python skills to combine these four arrays into a single, 4D NumPy array called <code>my4DArray</code>. Print its shape to confirm your dimensions are in the right order.</div>

### Saving Multidimensional TIF files

Like `imread`, the `scitkit-image` function `imsave` is able to deal with any multidimensional TIF file that `tiffile` is capable of saving - again, this means pretty much any TIF file that's compatabile with ImageJ/FIJI.

We can also include metadata in our TIF file, which ImageJ/FIJI can then read and use as appropriate.

In [ ]:
# Save our multicolour stack to a new ImageJ-compatible TIF file
# tifffile wants our array to be in TZCYXS order for imageJ compatibility
my4DArray = np.moveaxis(my4DArray,-1,1)  # so move C from the end to second dimension
io.imsave('./assets/bbbc034v1/AICS_12_134_C=all.tif',
          my4DArray,
          imagej=True,
          resolution=(1/x_pixel_size,1/y_pixel_size),
          metadata={'spacing':z_pixel_size,'unit':'nm'})

<div style="background-color:#abd9e9; border-radius: 5px; padding: 10pt"><strong>Task</strong> Open your new file in ImageJ/FIJI and check that the dimensions are correct and that the metadata has been read.</div>

## Cropping/Subsampling and NumPy Indexing

* Because we're usijng NumPy arrays to hold our image data, we can make use of all the benefits of NumPy Arrays
  * For example, we've already used `ndarray.max()` to calculate maximum intensity projections along an axis (dimension)
* One such advantage is NumPy indexing.
* Indexing allows us to crop our image, quickly subsampe our image (not the same as resizing properly), quickly extract the intensity for a list of pixels, e.g. pixels denoting a segmentented object.

In [ ]:
# Use a single plane from a 3D image array
mySlice =  c1[22,:,:]  # we've already seen this in our slice viewer above

# Plot
f, ax = plt.subplots(1,1)  # Create one subplot (1x1 grid)
ax.grid(False)
ax.imshow(mySlice, cmap='gray', interpolation='none')

plt.show()

In [ ]:
# Crop a 2D image in x and y
x_start = 256
y_start = 256
x_end = 768
y_end = 768

myCrop = mySlice[x_start:x_end,y_start:y_end]

# Plot
f, axes = plt.subplots(1,2)  # Create one subplot (1x2 grid)
(aS, aC) = axes.flatten()

aS.grid(False)
aS.imshow(mySlice, cmap='gray', interpolation='none')
aS.set_title('Full image')

aC.grid(False)
aC.imshow(myCrop, cmap='gray', interpolation='none')
aC.set_title('Cropped image')

plt.show()

In [ ]:
# Quickly subsample a 2D image by taking every fourth pixel in x and y
# Note we can resize more cleverly using interpolation methods
mySubsample = mySlice[::4,::4]

# Plot
f, axes = plt.subplots(1,2)  # Create one subplot (1x2 grid)
(aF, aS) = axes.flatten()

aF.grid(False)
aF.imshow(mySlice, cmap='gray', interpolation='none')
aF.set_title('Full image')

aS.grid(False)
aS.imshow(mySubsample, cmap='gray', interpolation='none')
aS.set_title('Subsampled image')

plt.show()

In [ ]:
# Explore the statistics of a list of pixels, e.g. a segmented area
myPixelIdx = np.meshgrid(np.arange(128,256),np.arange(512,768))
myPixelIdx = tuple(myPixelIdx)  # this future proofs us
myPixelVal = mySlice[myPixelIdx]

print('Region has a mean value of {0} and ranges between {1} and {2}.'.format(myPixelVal.mean(),myPixelVal.min(),myPixelVal.max()))

# Plot our image with the pixel's we investigated as white (actually use the region max).
myEditedSlice = mySlice.copy()
myEditedSlice[myPixelIdx] = myPixelVal.max()
f, ax = plt.subplots(1,1)  # Create one subplot (1x1 grid)
ax.grid(False)
ax.imshow(myEditedSlice, cmap='gray', interpolation='none')

plt.show()

## Histograms

* Finally, we often want to see the distribution of intensities for an image, channel or region of an image.
* This can be quckly accomplished using the plotting package [`seaborn`](https://seaborn.pydata.org).

In [ ]:
# Plot Each Channel (Z-MIP) and histogram
f, axes = plt.subplots(2,4)  # Create eight subplots (2x4 grid)
(i0, i1, i2, i3, h0, h1, h2, h3) = axes.flatten()

i0.imshow(c0.max(axis=0), cmap="gray", interpolation='none')
i0.set_axis_off()
i0.set_title("Channel 0 Z-MIP")

sns.distplot(c0.flatten(),kde=False,ax=h0)
h0.set_xlabel('Pixel Value')
h0.set_ylabel('Count')

i1.imshow(c1.max(axis=0), cmap="gray", interpolation='none')
i1.set_axis_off()
i1.set_title("Channel 1 Z-MIP")

sns.distplot(c1.flatten(),kde=False,ax=h1)
h1.set_xlabel('Pixel Value')
# h1.set_ylabel('Count')

i2.imshow(c2.max(axis=0), cmap="gray", interpolation='none')
i2.set_axis_off()
i2.set_title("Channel 2 Z-MIP")

sns.distplot(c2.flatten(),kde=False,ax=h2)
h2.set_xlabel('Pixel Value')
# h2.set_ylabel('Count')

i3.imshow(c3.max(axis=0), cmap="gray", interpolation='none')
i3.set_axis_off()
i3.set_title("Channel 3 Z-MIP")

sns.distplot(c3.flatten(),kde=False,ax=h3)
h3.set_xlabel('Pixel Value')
# h3.set_ylabel('Count')

plt.show()

## Summary

* `numpy` arrays can be used to store and manipulate digital images
  * The `shape` of a NumPy array corresponds to the image resolution
  * The `dtype` of a NumPy array corresponses to the bit-depth
  * We will use NumPy axes to correspond to image dimensions $t$, $z$, $x$, $y$, $\lambda$ in that order
  * Indexing allows us to access individual slices/channels, crop and sample from images
* We can use `scikit-image` to read and write multidimensional TIF files with `imread` and `imwrite`
* We can use `seaborn` to quickly plot a histogram (`distplot`) of our image/array